In [110]:
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

In [111]:
klek1 = pd.read_csv('./GRU_data/chembl_klek.csv')
klek2 = pd.read_csv('./GRU_data/250k_klek.csv')
klek = pd.concat([klek1, klek2])

selfies1 = pd.read_csv('./GRU_data/chembl_selfies.csv', names=['selfies'])
selfies2 = pd.read_csv('./GRU_data/250k_selfies.csv', names=['selfies'])
selfies = pd.concat([selfies1, selfies2])

In [125]:
klek2.shape

(183879, 2)

In [124]:
selfies2.shape

(183880, 1)